## Задание
1. Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:  
название источника;  
наименование новости;  
ссылку на новость;  
дата публикации.  
2. Сложить собранные данные в БД

In [1]:
import requests
from lxml import html
from pprint import pprint
from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client['News']
lenta = db.lenta
mail = db.mail

In [3]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) \
         Chrome/89.0.4389.90 Safari/537.36'}

In [4]:
def add_new_news(db, list):
    total = 0
    for i in list:
        if db.count_documents({'link': i['link']}) == 0:
            db.insert_one(i)
            total += 1
    print(f'Добавлено {total} новостей в {str(db)}')  

In [5]:
def lenta_news():
    lenta_news_list = []
    try:
        url = 'https://lenta.ru/'
        response = requests.get(url, headers=header)        
        dom = html.fromstring(response.text)
        
        items = dom.xpath("//section[contains(@class, 'b-top7-for-main')]/div/div[contains(@class, 'item')]//a[@href and ./time]")
        for item in items:
            top_news = {}

            name = item.xpath("./text()")
            link = item.xpath("./@href")
            date = item.xpath("./time/@datetime")

            top_news['source'] = 'lenta.ru'
            top_news['name'] = name[0].replace('\xa0', ' ')
            top_news['link'] = f"{url}{link[0]}"
            top_news['date'] = date
            lenta_news_list.append(top_news)
        return lenta_news_list
    except:
        print('Error')

In [6]:
pprint(lenta_news())

[{'date': [' 22:30, 12 апреля 2021'],
  'link': 'https://lenta.ru//news/2021/04/12/middd/',
  'name': 'МИД Турции прокомментировал решение России ограничить авиасообщение',
  'source': 'lenta.ru'},
 {'date': [' 23:43, 12 апреля 2021'],
  'link': 'https://lenta.ru//news/2021/04/12/smertnost/',
  'name': 'ВОЗ заявила о росте смертности от коронавируса в мире',
  'source': 'lenta.ru'},
 {'date': [' 23:39, 12 апреля 2021'],
  'link': 'https://lenta.ru//news/2021/04/12/schpartak/',
  'name': '«Спартак» захотел лишить «Локомотив» победы из-за сотрудника клуба '
          'в судейской',
  'source': 'lenta.ru'},
 {'date': [' 23:35, 12 апреля 2021'],
  'link': 'https://lenta.ru//news/2021/04/12/station/',
  'name': 'Россия построит собственную орбитальную станцию',
  'source': 'lenta.ru'},
 {'date': [' 23:25, 12 апреля 2021'],
  'link': 'https://lenta.ru//news/2021/04/12/poklonskaya/',
  'name': 'Поклонская попала под капельницу',
  'source': 'lenta.ru'},
 {'date': [' 23:14, 12 апреля 2021'],
 

In [7]:
def mail_news():
    mailru_news_list = []
    try:
        url = 'https://news.mail.ru/'
        response = requests.get(url, headers=header)
        dom = html.fromstring(response.text)
 
        items = dom.xpath("//div[contains(@class, 'js-module') and contains(@name, 'clb20268335') and contains(@data-module, 'TrackBlocks')]//div[contains(@class, 'daynews__item')] | //div[contains(@class, 'js-module') and contains(@name, 'clb20268335') and contains(@data-module, 'TrackBlocks')]/ul/li")
        for item in items:
            item_url = item.xpath("./a/@href")
            irem_response = requests.get(item_url[0])
            item_dom = html.fromstring(irem_response.text)
            top_news = {}

            source = item_dom.xpath(".//div[contains(@class, 'article js-article')]/*//span[contains(@class, 'link__text')]/text()")
            name = item_dom.xpath(".//div[contains(@class, 'article js-article')]/*[2]//h1[contains(@class, 'hdr__inner')]/text()")
            date = item_dom.xpath(".//div[contains(@class, 'article js-article')]/*//span[@datetime]/@datetime")

            top_news['source'] = source[0]
            top_news['name'] = name[0]
            top_news['link'] = item_url[0]
            top_news['date'] = date[0].replace('T', ' ')[:19]
            mailru_news_list.append(top_news)
        return mailru_news_list
    except:
        print('Error')

In [8]:
pprint(mail_news())

[{'date': '2021-04-12 19:29:55',
  'link': 'https://news.mail.ru/politics/45924032/',
  'name': 'Россия ограничивает регулярное и чартерное авиасообщение с Турцией '
          'с 15 апреля до 1 июня',
  'source': 'ТАСС'},
 {'date': '2021-04-12 21:35:21',
  'link': 'https://news.mail.ru/economics/45924900/',
  'name': '«Огромный удар по отрасли»: туроператоры оценили стоимость '
          'забронированных туров в Турцию',
  'source': 'Forbes'},
 {'date': '2021-04-12 19:11:22',
  'link': 'https://news.mail.ru/incident/45923709/',
  'name': 'Огонь с «Невской мануфактуры» перекинулся на соседнее здание',
  'source': 'Коммерсантъ'},
 {'date': '2021-04-12 20:08:04',
  'link': 'https://news.mail.ru/politics/45924370/',
  'name': 'Зеленский призвал Байдена помочь Украине вступить в НАТО «не только '
          'на словах»',
  'source': 'Lenta.Ru'},
 {'date': '2021-04-12 23:17:48',
  'link': 'https://news.mail.ru/society/45923743/',
  'name': 'Портрет Гагарина появился на самой высокой башне мир

In [9]:
print(add_new_news(lenta, lenta_news()))
print(add_new_news(mail, mail_news()))

Добавлено 10 новостей в Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'News'), 'lenta')
None
Добавлено 11 новостей в Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'News'), 'mail')
None


In [10]:
# mail.delete_many({})
# lenta.delete_many({})